In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

GLOUD_CREDENTIALS = "<path to credentials.json>"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GLOUD_CREDENTIALS

In [2]:
import pandas as pd

test_df = pd.read_csv("persona_identification_testset.csv", index_col="index")
test_df

,character_name,movie_name,persona,movie_summary
index,,,,
5,Ellen Aim,Streets of Fire,chanteuse,"In an unnamed city, Ellen Aim , lead singer of..."
15,Timothy,The Long Kiss Goodnight,casanova,Samantha Caine is a mother and schoolteacher ...
16,Johnny Wickham,Bride and Prejudice,casanova,The plot of the movie is based on Austen′s nov...
26,Roy O'Bannon,Shanghai Noon,retired_outlaw,Chon Wang is a Chinese imperial guard. After P...
36,Jackson Rippner,Red Eye,psycho_for_hire,After attending her grandmother's funeral in D...
...,...,...,...,...
496,Lt. Commander Ron Hunter,Crimson Tide,father_to_his_men,"In post-Soviet Russia, military units loyal to..."
497,Christopher Pike,Star Trek,father_to_his_men,"In 2233, the Federation starship USS Kelvin is..."
498,Zordon,Mighty Morphin' Power Rangers: The Movie,father_to_his_men,"The Power Rangers, Adam , Kimberly , Billy , A..."


In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join("..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from src.models.persona_identification import PersonaIdentification
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain_redis import RedisCache

# cache = InMemoryCache()
cache = RedisCache(redis_url="redis://localhost:6379")
set_llm_cache(cache)

rate_limiter = InMemoryRateLimiter(
    requests_per_second=4.99 / 60,  # free google cloud account allows 5 requests per minute
    check_every_n_seconds=0.1,
    max_bucket_size=5,
)

pi = PersonaIdentification(
    config_path="../../src/models/person_identification_config.yaml",
    cache=cache,
    rate_limiter=rate_limiter,
)

test_df_results = pi.batch(test_df.to_dict(orient="records"))

In [28]:
test_df_results[0]

AIMessage(content='Chanteuse', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 1331, 'candidates_token_count': 4, 'total_token_count': 1335, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.005151043180376291, 'logprobs_result': {'top_candidates': [], 'chosen_candidates': []}, 'parsing_success': True}, id='run-efcef1dc-d731-4207-9e99-368a422cc09c-0', usage_metadata={'input_tokens': 1331, 'output_tokens': 4, 'total_tokens': 1335})

In [25]:
from src.models.persona_identification import persona_lowercase

test_df["model_predictions"] = [persona_lowercase(res.content) for res in test_df_results]
test_df["is_prediction_valid"] = [1 if res.response_metadata["parsing_success"] else 0 for res in test_df_results]

print("Not valid predictions:")
display(test_df[test_df.is_prediction_valid == 0])
print(f"Prediction accuracy: {(test_df.model_predictions == test_df.persona).mean(): .3f}")
test_df.to_excel("model_predictions_analysis.xlsx");

Not valid predictions:


,character_name,movie_name,persona,movie_summary,model_predictions,is_prediction_valid
index,,,,,,
322,Michael Newman,Click,ditz,"Michael Newman , an architect, is married to h...",workaholic\n\n,0
448,The Lieutenant,Bad Lieutenant,dirty_cop,"The film opens in the Bronx, where The Lieuten...",,0


Prediction accuracy:  0.426
